In [19]:
import datetime
import logging
import os
from subprocess import call

import numpy as np
import pandas as pd
from sqlalchemy import create_engine


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# загружаем данные в Postgres
call("bash /home/Egorov_load_project.sh", shell=True)

engine = create_engine('postgresql://postgres:@{}@postgres_host')

In [49]:
# загрузка данных из Postgres в датафрейм pandas
accounts = pd.read_sql('select * from public.v_accounts ', engine)
logger.info("select из общего view со всеми атрибутами")
print(accounts.head())
logger.info("Загрузили типы данных {}".format(ratings.dtypes))

2018-08-24 10:50:12,578 : INFO : select из общего view со всеми атрибутами
2018-08-24 10:50:12,589 : INFO : Загрузили типы данных Series([], dtype: object)


         account_number   account_name   client_name report_date  acc_value  \
0  42301000000000000001  Счет депозита   Иванов А.С.  2015-01-01   -10000.0   
1  42301000000000000002  Счет депозита  Смирнов А.С.  2015-01-01   -50000.0   
2  45201000000000000003  Счет депозита  Соболев А.С.  2015-01-01    -1000.0   
3  45201000000000000004   Ссудный счет   Иванов А.С.  2015-01-01   580000.0   
4  45201000000000000005   Ссудный счет  Смирнов А.С.  2015-01-01   400000.0   

  department_name  
0      Московский  
1  Северозападный  
2      Московский  
3      Московский  
4  Северозападный  


In [155]:
#Подсчет счетов у клиента аналог SQL запроса № 5

account_count = accounts.groupby(['client_name']).agg({'acc_value':['count']})#.filter(lambda x: len(x) >= 3)


In [156]:
print(account_count.head())

             acc_value
                 count
client_name           
Иванов А.С.          2
Смирнов А.С.         2
Соболев А.С.         1


In [159]:
#Вывод клиентов только с положительным балансом - аналог SQL запроса №9

window_sums = accounts.assign( 
    cumsum =accounts.groupby(['client_name'])['acc_value']
    #.agg('sum') 
    .cumsum()# .rank(method='dense', ascending=True).astype(int) 
)   .query('cumsum >0').sort_values(['client_name', 'acc_value'])
print(window_sums.head())

#pd.merge(left,right,on=['id','subject_id'])

         account_number  account_name   client_name report_date  acc_value  \
3  45201000000000000004  Ссудный счет   Иванов А.С.  2015-01-01   580000.0   
4  45201000000000000005  Ссудный счет  Смирнов А.С.  2015-01-01   400000.0   

  department_name    cumsum  
3      Московский  570000.0  
4  Северозападный  350000.0  


In [157]:
account_count.to_csv("account_count.csv", sep='\t', encoding='utf-8')

In [160]:
window_sums.apply(lambda x: x).to_csv("window_sums.csv", sep='\t', encoding='utf-8')